# Additional End of week Exercise - week 2

Now use everything you've learned from Week 2 to build a full prototype for the technical question/answerer you built in Week 1 Exercise.

This should include a Gradio UI, streaming, use of the system prompt to add expertise, and the ability to switch between models. Bonus points if you can demonstrate use of a tool!

If you feel bold, see if you can add audio input so you can talk to it, and have it respond with audio. ChatGPT or Claude can help you, or email me if you have questions.

I will publish a full solution here soon - unless someone beats me to it...

There are so many commercial applications for this, from a language tutor, to a company onboarding solution, to a companion AI to a course (like this one!) I can't wait to see your results.

In [1]:
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
import requests
import os


In [2]:
load_dotenv()

True

In [3]:
# Initialize clients
openai_client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
ollama_client = OpenAI(base_url="http://localhost:11434/v1", api_key='ollama')

# Available models
OPENAI_MODELS = ['gpt-4o-mini', 'gpt-4o', 'gpt-3.5-turbo']
OLLAMA_MODELS = ['llama3.2', 'deepseek-r1:1.5b', 'codellama']


In [4]:
# System prompts with different expertise levels
SYSTEM_PROMPTS = {
    "Code Analyzer": """You are an expert code analyst. Analyze code clearly and concisely.
Explain what each section does, identify potential issues, and suggest improvements.
Respond in markdown format.""",
    
    "Beginner Tutor": """You are a patient programming tutor for beginners.
Explain code in simple terms, use analogies, and break down complex concepts.
Encourage learning and provide helpful examples.""",
    
    "Senior Developer": """You are a senior software engineer conducting a code review.
Focus on architecture, design patterns, performance, security, and best practices.
Be direct and technical in your feedback.""",
    
    "Debug Assistant": """You are a debugging expert. Analyze code for bugs, edge cases,
and potential runtime errors. Suggest fixes and explain why issues occur."""
}

In [5]:
def check_ollama_connection():
    """Check if Ollama is running"""
    try:
        response = requests.get("http://localhost:11434")
        return response.status_code == 200
    except:
        return False

In [6]:
def get_client_and_model(provider, model):
    """Get the appropriate client based on provider"""
    if provider == "OpenAI":
        return openai_client, model
    else:  # Ollama
        if not check_ollama_connection():
            raise Exception("Ollama is not running. Please start Ollama service.")
        return ollama_client, model


In [7]:
def analyze_code_stream(code_text, provider, model, expertise, temperature):
    """Analyze code with streaming response"""
    try:
        client, model_name = get_client_and_model(provider, model)
        system_prompt = SYSTEM_PROMPTS[expertise]
        
        user_prompt = f"Here is the code to analyze:\n\n{code_text}\n\nPlease provide a detailed analysis."

        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]

        stream = client.chat.completions.create(
            model=model_name,
            messages=messages,
            temperature=temperature,
            stream=True
        )

        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                full_response += content
                yield full_response

    except Exception as e:
        yield f"Error: {str(e)}"


def transcribe_audio(audio_path):
    """Transcribe audio to text using OpenAI Whisper"""
    try:
        with open(audio_path, "rb") as audio_file:
            transcript = openai_client.audio.transcriptions.create(
                model="whisper-1",
                file=audio_file
            )
        return transcript.text
    except Exception as e:
        return f"Error transcribing audio: {str(e)}"


def text_to_speech(text):
    """Convert text to speech using OpenAI TTS"""
    try:
        response = openai_client.audio.speech.create(
            model="tts-1",
            voice="alloy",
            input=text[:4096]  # Limit text length
        )
        
        # Save to temporary file
        output_path = "output_speech.mp3"
        response.stream_to_file(output_path)
        return output_path
    except Exception as e:
        return None


def process_audio_input(audio, provider, model, expertise, temperature):
    """Process audio input and return both text analysis and audio response"""
    if audio is None:
        return "Please record or upload audio.", None
    
    # Transcribe audio
    transcribed_text = transcribe_audio(audio)
    
    if transcribed_text.startswith("Error"):
        return transcribed_text, None
    
    # Get analysis (non-streaming for audio response)
    try:
        client, model_name = get_client_and_model(provider, model)
        system_prompt = SYSTEM_PROMPTS[expertise]
        
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Here is the code to analyze:\n\n{transcribed_text}"}
        ]

        response = client.chat.completions.create(
            model=model_name,
            messages=messages,
            temperature=temperature
        )
        
        analysis = response.choices[0].message.content
        
        # Convert response to speech
        audio_output = text_to_speech(analysis)
        
        return f"**Transcribed Code:**\n```\n{transcribed_text}\n```\n\n**Analysis:**\n{analysis}", audio_output
    
    except Exception as e:
        return f"Error: {str(e)}", None



In [8]:
# Create Gradio Interface
with gr.Blocks(theme=gr.themes.Soft(), title="Technical Code Analyzer") as app:
    gr.Markdown("""
    # 🔍 Technical Code Analyzer
    ### Week 2 Exercise - Full Featured Prototype
    Analyze code with AI assistance using text or voice input!
    """)
    
    with gr.Tabs():
        # Tab 1: Text Input
        with gr.Tab("📝 Text Analysis"):
            with gr.Row():
                with gr.Column(scale=1):
                    provider_text = gr.Radio(
                        choices=["OpenAI", "Ollama"],
                        value="OpenAI",
                        label="Provider"
                    )
                    
                    model_text = gr.Dropdown(
                        choices=OPENAI_MODELS,
                        value="gpt-4o-mini",
                        label="Model"
                    )
                    
                    expertise_text = gr.Dropdown(
                        choices=list(SYSTEM_PROMPTS.keys()),
                        value="Code Analyzer",
                        label="Expertise Level"
                    )
                    
                    temperature_text = gr.Slider(
                        minimum=0,
                        maximum=1,
                        value=0,
                        step=0.1,
                        label="Temperature"
                    )
                
                with gr.Column(scale=2):
                    code_input = gr.Code(
                        label="Enter Code to Analyze",
                        language="python",
                        lines=15
                    )
                    
                    analyze_btn = gr.Button("🚀 Analyze Code", variant="primary")
            
            output_text = gr.Markdown(label="Analysis")
            
            # Example codes
            gr.Examples(
                examples=[
                    ["""def fetch_website_links(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    links = [link.get("href") for link in soup.find_all("a")]
    return [link for link in links if link]"""],
                    ["""def quicksort(arr):
    if len(arr) <= 1:
        return arr
    pivot = arr[len(arr) // 2]
    left = [x for x in arr if x < pivot]
    middle = [x for x in arr if x == pivot]
    right = [x for x in arr if x > pivot]
    return quicksort(left) + middle + quicksort(right)"""]
                ],
                inputs=code_input
            )
            
            # Update model choices based on provider
            def update_models(provider):
                if provider == "OpenAI":
                    return gr.Dropdown(choices=OPENAI_MODELS, value="gpt-4o-mini")
                else:
                    return gr.Dropdown(choices=OLLAMA_MODELS, value="llama3.2")
            
            provider_text.change(
                fn=update_models,
                inputs=provider_text,
                outputs=model_text
            )
            
            analyze_btn.click(
                fn=analyze_code_stream,
                inputs=[code_input, provider_text, model_text, expertise_text, temperature_text],
                outputs=output_text
            )
        
        # Tab 2: Audio Input
        with gr.Tab("🎤 Voice Analysis"):
            gr.Markdown("""
            ### Speak or upload your code question
            The system will transcribe your voice, analyze the code, and respond with both text and audio!
            """)
            
            with gr.Row():
                with gr.Column(scale=1):
                    provider_audio = gr.Radio(
                        choices=["OpenAI", "Ollama"],
                        value="OpenAI",
                        label="Provider"
                    )
                    
                    model_audio = gr.Dropdown(
                        choices=OPENAI_MODELS,
                        value="gpt-4o-mini",
                        label="Model"
                    )
                    
                    expertise_audio = gr.Dropdown(
                        choices=list(SYSTEM_PROMPTS.keys()),
                        value="Code Analyzer",
                        label="Expertise Level"
                    )
                    
                    temperature_audio = gr.Slider(
                        minimum=0,
                        maximum=1,
                        value=0,
                        step=0.1,
                        label="Temperature"
                    )
                
                with gr.Column(scale=2):
                    audio_input = gr.Audio(
                        sources=["microphone", "upload"],
                        type="filepath",
                        label="Record or Upload Audio"
                    )
                    
                    analyze_audio_btn = gr.Button("🎯 Analyze from Audio", variant="primary")
            
            output_audio_text = gr.Markdown(label="Analysis")
            output_audio_speech = gr.Audio(label="Audio Response")
            
            provider_audio.change(
                fn=update_models,
                inputs=provider_audio,
                outputs=model_audio
            )
            
            analyze_audio_btn.click(
                fn=process_audio_input,
                inputs=[audio_input, provider_audio, model_audio, expertise_audio, temperature_audio],
                outputs=[output_audio_text, output_audio_speech]
            )
        
        # Tab 3: About
        with gr.Tab("ℹ️ About"):
            gr.Markdown("""
            ## Features
            
            ✅ **Multiple AI Providers**: Switch between OpenAI and Ollama models
            
            ✅ **Streaming Responses**: See analysis appear in real-time
            
            ✅ **Custom Expertise Levels**: Choose different system prompts for various use cases
            
            ✅ **Text & Voice Input**: Type code or speak it naturally
            
            ✅ **Audio Output**: Get responses in both text and speech
            
            ✅ **Temperature Control**: Adjust creativity of responses
            
            ## Setup Instructions
            
            ### Requirements:
            ```bash
            pip install gradio openai python-dotenv requests
            ```
            
            ### Environment Variables:
            Create a `.env` file with:
            ```
            OPENAI_API_KEY=your_api_key_here
            ```
            
            ### For Ollama:
            1. Install Ollama from https://ollama.ai
            2. Run: `ollama pull llama3.2`
            3. Run: `ollama pull deepseek-r1:1.5b`
            4. Start Ollama service
            
            ## Use Cases
            
            - 📚 **Learning Tool**: Understand complex code snippets
            - 🔍 **Code Review**: Get expert feedback on your code
            - 🐛 **Debugging**: Find and fix issues
            - 🎓 **Education**: Perfect for courses and tutorials
            - 🏢 **Onboarding**: Help new developers understand codebases
            """)

# Launch the app
if __name__ == "__main__":
    app.launch(share=False)

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
